In [8]:
import camelot
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
koridor_data = {}

for i in range(1, 15):
    filename = f"Jumlah Pelanggan BRT Transjakarta Perhalte Koridor {i} Tahun 2024.pdf"
    try:
        tables = camelot.read_pdf(filename, pages="all")
        df = tables[0].df
        df = df.drop(df.columns[0], axis=1)
        df.columns = df.iloc[1]
        df = df.iloc[2:].reset_index(drop=True)
        df = df[df[df.columns[0]].str.strip().astype(bool)]  # Remove empty rows
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col].str.replace('.', ''), errors='coerce')
        koridor_data[f"Koridor{i}"] = df
        print(f"✓ Loaded Koridor {i}: {len(df)} stops")
    except Exception as e:
        print(f"✗ Failed to load Koridor {i}: {e}")

print(f"✓ Total loaded corridors: {len(koridor_data)}")

✓ Loaded Koridor 1: 30 stops
✓ Loaded Koridor 2: 24 stops
✓ Loaded Koridor 3: 13 stops
✓ Loaded Koridor 4: 17 stops
✓ Loaded Koridor 5: 17 stops
✓ Loaded Koridor 6: 18 stops
✓ Loaded Koridor 7: 14 stops
✓ Loaded Koridor 8: 22 stops
✓ Loaded Koridor 9: 33 stops
✓ Loaded Koridor 10: 18 stops
✓ Loaded Koridor 11: 15 stops
✓ Loaded Koridor 12: 14 stops
✓ Loaded Koridor 13: 15 stops
✓ Loaded Koridor 14: 5 stops
✓ Total loaded corridors: 14


In [10]:
# ==================== CORRIDOR SPECIFICATIONS ====================
corridor_specs = {
    1: {'length_km': 12.9, 'stations': 30},
    2: {'length_km': 14.0, 'stations': 24},
    3: {'length_km': 19.0, 'stations': 13},
    4: {'length_km': 11.85, 'stations': 17},
    5: {'length_km': 13.5, 'stations': 17},
    6: {'length_km': 13.3, 'stations': 18},
    7: {'length_km': 12.8, 'stations': 14},
    8: {'length_km': 26.0, 'stations': 22},
    9: {'length_km': 28.8, 'stations': 33},
    10: {'length_km': 19.4, 'stations': 18},
    11: {'length_km': 11.4, 'stations': 15},
    12: {'length_km': 23.8, 'stations': 14},
    13: {'length_km': 9.4, 'stations': 15},
    14: {'length_km': 9.7, 'stations': 5}
} 

In [11]:
# ==================== OPTIMIZATION CLASS ====================
class TransjakartaSystemOptimizer:
    def __init__(self, koridor_data, corridor_specs):
        self.koridor_data = koridor_data
        self.corridor_specs = corridor_specs
        self.results = {}
        
    def process_corridor_data(self, corridor_name):
        """Process passenger data for a specific corridor"""
        if corridor_name not in self.koridor_data:
            return None
            
        df = self.koridor_data[corridor_name].copy()
        month_cols = [col for col in df.columns if col != df.columns[0]]
        
        df['Avg_Monthly'] = df[month_cols].mean(axis=1)
        df['Peak_Month'] = df[month_cols].max(axis=1)
        df['Daily_Avg'] = df['Avg_Monthly'] / 30
        df['Peak_Hour'] = df['Daily_Avg'] * 0.15
        
        return df.dropna(subset=['Avg_Monthly']).reset_index(drop=True)
    
    def calculate_cycle_time(self, corridor_num, avg_speed=25, dwell_time=0.5, layover=10):
        """Calculate round trip cycle time"""
        specs = self.corridor_specs[corridor_num]
        length = specs['length_km']
        travel_time = (length * 2 * 60) / avg_speed
        total_dwell = dwell_time * specs['stations'] * 2
        return travel_time + total_dwell + layover
    
    
    def optimize_corridor(self, corridor_num, vehicle_capacity=103, load_factor=0.85):
        """Optimize fleet for specific corridor"""
        corridor_name = f"Koridor{corridor_num}"
        df = self.process_corridor_data(corridor_name)
        
        if df is None:
            return None
            
        # Calculate demand
        total_monthly = df['Avg_Monthly'].sum()
        peak_demand = df['Peak_Hour'].sum()
        
        # Get specs and cycle time
        specs = self.corridor_specs[corridor_num]
        cycle_time = self.calculate_cycle_time(corridor_num)
        
        # Optimization scenarios
        effective_capacity = vehicle_capacity * load_factor
        min_headway = max((effective_capacity * 60) / peak_demand, 3) if peak_demand > 0 else 10
        
        scenarios = {
            'Current': 10,
            'Optimal': min_headway,
            'Balanced': max(7, min_headway * 0.7),
            'Frequent': 5,
            'Cost_Efficient': 12
        }
        
        scenario_results = {}
        for name, headway in scenarios.items():
            fleet_size = np.ceil(cycle_time / headway) 
            total_fleet = fleet_size * 1.2 # For operational provisions
            
            buses_per_hour = 60 / headway
            hourly_capacity = buses_per_hour * effective_capacity
            utilization = (peak_demand / hourly_capacity * 100) if hourly_capacity > 0 else 0
            
            # Cost calculation
            fuel_per_bus_day = (specs['length_km'] * 2 * 8) / 3.5  #8 trips, 3.5km/L
            fuel_cost = total_fleet * fuel_per_bus_day * 6800  #IDR 6.8k/L (BioSolar)
            driver_cost = total_fleet * 550000  # 1 driver + 1 guide @ IDR 16.5 Million/month each
            maintenance_cost = total_fleet * 530000 #530k IDR average/day
            daily_cost = fuel_cost + driver_cost + maintenance_cost
            
            scenario_results[name] = {
                'headway': headway,
                'fleet_size': fleet_size,
                'total_fleet': total_fleet,
                'utilization': utilization,
                'daily_cost': daily_cost,
                'hourly_capacity': hourly_capacity,
                'fuel_liters': total_fleet * fuel_per_bus_day
            }
        
        return {
            'corridor': corridor_num,
            'specs': specs,
            'demand': {
                'monthly_total': total_monthly,
                'daily_total': total_monthly / 30,
                'peak_hour': peak_demand,
                'active_stops': len(df)
            },
            'scenarios': scenario_results,
            'passenger_data': df,
            'cycle_time': cycle_time
        }
    
    def analyze_hotspots(self, corridor_num, top_n=10):
        """Identify passenger hotspots"""
        corridor_name = f"Koridor{corridor_num}"
        df = self.process_corridor_data(corridor_name)
        
        if df is None:
            return []
            
        return df.nlargest(top_n, 'Avg_Monthly')[
            [df.columns[0], 'Avg_Monthly', 'Daily_Avg', 'Peak_Hour']
        ].to_dict('records')
    
    def recommend_new_stops(self, corridor_num, top_n=5):
        """Recommend new bus stop locations"""
        corridor_name = f"Koridor{corridor_num}"
        df = self.process_corridor_data(corridor_name)
        
        if df is None or len(df) < 2:
            return []
            
        recommendations = []
        for i in range(len(df) - 1):
            current = df.iloc[i]
            next_stop = df.iloc[i + 1]
            
            potential_demand = (current['Avg_Monthly'] + next_stop['Avg_Monthly']) * 0.3
            
            recommendations.append({
                'location': f"Between {current[df.columns[0]]} - {next_stop[df.columns[0]]}",
                'potential_monthly': potential_demand,
                'priority_score': potential_demand,
                'current_stop': current[df.columns[0]],
                'next_stop': next_stop[df.columns[0]]
            })
        
        recommendations.sort(key=lambda x: x['potential_monthly'], reverse=True)
        return recommendations[:top_n]
    
    def analyze_all_corridors(self):
        """Analyze all loaded corridors"""
        all_results = {}
        summary_data = []
        
        for corridor_name in self.koridor_data.keys():
            corridor_num = int(corridor_name.replace('Koridor', ''))
            result = self.optimize_corridor(corridor_num)
            
            if result:
                all_results[corridor_num] = result
                balanced = result['scenarios']['Balanced']
                
                summary_data.append({
                    'Corridor': corridor_num,
                    'Length_km': result['specs']['length_km'],
                    'Active_Stops': result['demand']['active_stops'],
                    'Monthly_Passengers': result['demand']['monthly_total'],
                    'Peak_Hour_Demand': result['demand']['peak_hour'],
                    'Recommended_Fleet': balanced['total_fleet'],
                    'Recommended_Headway': balanced['headway'],
                    'Daily_Cost_IDR': balanced['daily_cost'],
                    'Utilization_pct': balanced['utilization'],
                    'Daily_Fuel_L': balanced['fuel_liters']
                })
        
        return all_results, pd.DataFrame(summary_data) if summary_data else None

    def print_corridor_analysis(self, corridor_num):
        """Print detailed analysis for a specific corridor"""
        print(f"\n{'='*20} KORIDOR {corridor_num} ANALYSIS {'='*20}")
        
        result = self.optimize_corridor(corridor_num)
        if result is None:
            print(f"No data available for Koridor {corridor_num}")
            return
            
        # Basic info
        print(f"Corridor Length: {result['specs']['length_km']} km")
        print(f"Active Stops: {result['demand']['active_stops']}")
        print(f"Monthly Passengers: {result['demand']['monthly_total']:,.0f}")
        print(f"Peak Hour Demand: {result['demand']['peak_hour']:,.0f} passengers/hour")
        
        # Scenario comparison
        print(f"\nSCENARIO COMPARISON:")
        print(f"{'Scenario':<15} {'Headway':<8} {'Fleet':<6} {'Util%':<8} {'Daily Cost':<12}")
        print("-" * 60)
        
        recommended_scenario = None
        best_score = 0
        
        for scenario, data in result['scenarios'].items():
            utilization = data['utilization']
            cost_millions = data['daily_cost'] / 1000000
            
            print(f"{scenario:<15} {data['headway']:<8.1f} {data['total_fleet']:<6.0f} "
                  f"{utilization:<8.1f} IDR {cost_millions:<8.1f}M")
            
            # Find best scenario (reasonable utilization + cost efficiency)
            if 60 <= utilization <= 150:
                efficiency_score = 100 - abs(utilization - 100)
                cost_score = max(0, 100 - cost_millions * 5)
                total_score = efficiency_score + cost_score
                
                if total_score > best_score:
                    best_score = total_score
                    recommended_scenario = scenario
        
        # Recommendations
        if recommended_scenario:
            rec_data = result['scenarios'][recommended_scenario]
            print(f"\nRECOMMENDED SCENARIO: {recommended_scenario}")
            print(f"   Fleet Size: {rec_data['total_fleet']:.0f} buses")
            print(f"   Headway: {rec_data['headway']:.0f} minutes")
            print(f"   Utilization: {rec_data['utilization']:.1f}%")
            print(f"   Daily Cost: IDR {rec_data['daily_cost']/1000000:.1f} million")
            print(f"   Daily Fuel: {rec_data['fuel_liters']:.0f} liters")
        
        # Hotspots
        hotspots = self.analyze_hotspots(corridor_num, 5)
        if hotspots:
            print(f"\nTOP 5 PASSENGER HOTSPOTS:")
            for i, stop in enumerate(hotspots, 1):
                print(f"   {i}. {stop[result['passenger_data'].columns[0]]}: {stop['Avg_Monthly']:,.0f}/month")
        
        # New stop recommendations
        new_stops = self.recommend_new_stops(corridor_num, 3)
        if new_stops:
            print(f"\nTOP 3 NEW STOP RECOMMENDATIONS:")
            for i, rec in enumerate(new_stops, 1):
                print(f"   {i}. {rec['location']}")
                print(f"      Potential: {rec['potential_monthly']:,.0f} passengers/month")

# ==================== ANALYSIS EXECUTION ====================
# Initialize optimizer
optimizer = TransjakartaSystemOptimizer(koridor_data, corridor_specs)

# Analyze all corridors
print("\nAnalyzing all loaded corridors...")
all_results, summary_df = optimizer.analyze_all_corridors()

if summary_df is not None:
    print("\n" + "="*80)
    print("SYSTEM-WIDE OPTIMIZATION SUMMARY")
    print("="*80)
    print(summary_df.to_string(index=False))
    
    # System totals
    total_passengers = summary_df['Monthly_Passengers'].sum()
    total_fleet = summary_df['Recommended_Fleet'].sum()
    total_cost = summary_df['Daily_Cost_IDR'].sum()
    total_fuel = summary_df['Daily_Fuel_L'].sum()
    
    print(f"\nSYSTEM TOTALS:")
    print(f"Total Monthly Passengers: {total_passengers:,.0f}")
    print(f"Total Recommended Fleet: {total_fleet:.0f} buses")
    print(f"Total Daily Operating Cost: IDR {total_cost/1000000:.1f} million")
    print(f"Total Daily Fuel Consumption: {total_fuel:,.0f} liters")
    print(f"Average Cost per Passenger: IDR {total_cost*30/total_passengers:.0f}")

# Print detailed analysis for each corridor
for corridor_name in koridor_data.keys():
    corridor_num = int(corridor_name.replace('Koridor', ''))
    optimizer.print_corridor_analysis(corridor_num)




Analyzing all loaded corridors...

SYSTEM-WIDE OPTIMIZATION SUMMARY
 Corridor  Length_km  Active_Stops  Monthly_Passengers  Peak_Hour_Demand  Recommended_Fleet  Recommended_Headway  Daily_Cost_IDR  Utilization_pct  Daily_Fuel_L
        1      12.90            23        1.738658e+06       8693.289167               18.0             7.000000    2.665810e+07      1158.443255   1061.485714
        2      14.00            22        9.351685e+05       4675.842500               18.0             7.000000    2.727360e+07       623.089615   1152.000000
        3      19.00            13        8.808870e+05       4404.435000               20.4             7.000000    3.408082e+07       586.922616   1771.885714
        4      11.85            17        5.265954e+05       2632.977083               14.4             7.000000    2.085647e+07       350.863118    780.068571
        5      13.50            17        8.629871e+05       4314.935417               16.8             7.000000    2.519424e+07   

In [12]:
# ==================== EXPORT RESULTS ====================
def export_all_results():
    """Export all analysis results to CSV files"""
    export_files = []
    
    # System summary
    if summary_df is not None:
        summary_df.to_csv('transjakarta_system_optimization.csv', index=False)
        export_files.append('transjakarta_system_optimization.csv')
    
    # Individual corridor results
    for corridor_num, result in all_results.items():
        # Passenger data
        result['passenger_data'].to_csv(f'koridor{corridor_num}_passenger_data.csv', index=False)
        
        # Scenarios
        scenarios_df = pd.DataFrame([
            {'Scenario': k, **v} for k, v in result['scenarios'].items()
        ])
        scenarios_df.to_csv(f'koridor{corridor_num}_scenarios.csv', index=False)
        
        # Hotspots
        hotspots = optimizer.analyze_hotspots(corridor_num, 10)
        pd.DataFrame(hotspots).to_csv(f'koridor{corridor_num}_hotspots.csv', index=False)
        
        # New stops
        new_stops = optimizer.recommend_new_stops(corridor_num, 8)
        pd.DataFrame(new_stops).to_csv(f'koridor{corridor_num}_new_stops.csv', index=False)
        
        export_files.extend([
            f'koridor{corridor_num}_passenger_data.csv',
            f'koridor{corridor_num}_scenarios.csv',
            f'koridor{corridor_num}_hotspots.csv',
            f'koridor{corridor_num}_new_stops.csv'
        ])
    
    print(f"\nExported {len(export_files)} files:")
    for file in export_files:
        print(f"   - {file}")
    
    return export_files

# Export all results
exported_files = export_all_results()

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print("All corridors have been optimized with comprehensive recommendations.")
print("Check the exported CSV files for detailed data and analysis results.")


Exported 57 files:
   - transjakarta_system_optimization.csv
   - koridor1_passenger_data.csv
   - koridor1_scenarios.csv
   - koridor1_hotspots.csv
   - koridor1_new_stops.csv
   - koridor2_passenger_data.csv
   - koridor2_scenarios.csv
   - koridor2_hotspots.csv
   - koridor2_new_stops.csv
   - koridor3_passenger_data.csv
   - koridor3_scenarios.csv
   - koridor3_hotspots.csv
   - koridor3_new_stops.csv
   - koridor4_passenger_data.csv
   - koridor4_scenarios.csv
   - koridor4_hotspots.csv
   - koridor4_new_stops.csv
   - koridor5_passenger_data.csv
   - koridor5_scenarios.csv
   - koridor5_hotspots.csv
   - koridor5_new_stops.csv
   - koridor6_passenger_data.csv
   - koridor6_scenarios.csv
   - koridor6_hotspots.csv
   - koridor6_new_stops.csv
   - koridor7_passenger_data.csv
   - koridor7_scenarios.csv
   - koridor7_hotspots.csv
   - koridor7_new_stops.csv
   - koridor8_passenger_data.csv
   - koridor8_scenarios.csv
   - koridor8_hotspots.csv
   - koridor8_new_stops.csv
   - korid

In [13]:
# Analyze specific corridor
optimizer.print_corridor_analysis(1)  # Detailed analysis for Koridor 1

# Get hotspots for specific corridor
hotspots = optimizer.analyze_hotspots(1, top_n=10)

# Get new stop recommendations
new_stops = optimizer.recommend_new_stops(1, top_n=5)

# Access raw optimization results
result = optimizer.optimize_corridor(1)
print(result['scenarios']['Balanced'])  # Get balanced scenario data

# Access passenger data
df = optimizer.process_corridor_data('Koridor1')
print(df.head())  # View processed passenger data


==================== KORIDOR 1 ANALYSIS ====================
Corridor Length: 12.9 km
Active Stops: 23
Monthly Passengers: 1,738,658
Peak Hour Demand: 8,693 passengers/hour

SCENARIO COMPARISON:
Scenario        Headway  Fleet  Util%    Daily Cost  
------------------------------------------------------------
Current         10.0     13     1654.9   IDR 19.5    M
Optimal         3.0      41     496.5    IDR 60.4    M
Balanced        7.0      18     1158.4   IDR 26.7    M
Frequent        5.0      25     827.5    IDR 37.3    M
Cost_Efficient  12.0     11     1985.9   IDR 16.0    M

TOP 5 PASSENGER HOTSPOTS:
   1. GELORA BUNG KARNO: 193,198/month
   2. BUNDARAN HI ASTRA: 183,935/month
   3. BLOK M: 126,158/month
   4. KARET: 125,628/month
   5. BUNDARAN SENAYAN: 125,212/month

TOP 3 NEW STOP RECOMMENDATIONS:
   1. Between BUNDARAN SENAYAN - GELORA BUNG KARNO
      Potential: 95,523 passengers/month
   2. Between TELUK BETUNG - BUNDARAN HI ASTRA
      Potential: 82,949 passengers/month
   

In [14]:
# Analyze specific corridor
optimizer.print_corridor_analysis(13)  # Detailed analysis for Koridor 1

# Get hotspots for specific corridor
hotspots = optimizer.analyze_hotspots(13, top_n=10)

# Get new stop recommendations
new_stops = optimizer.recommend_new_stops(13, top_n=5)

# Access raw optimization results
result = optimizer.optimize_corridor(13)
print(result['scenarios']['Balanced'])  # Get balanced scenario data

# Access passenger data
df = optimizer.process_corridor_data('Koridor13')
print(df.head())  # View processed passenger data


==================== KORIDOR 13 ANALYSIS ====================
Corridor Length: 9.4 km
Active Stops: 15
Monthly Passengers: 1,115,862
Peak Hour Demand: 5,579 passengers/hour

SCENARIO COMPARISON:
Scenario        Headway  Fleet  Util%    Daily Cost  
------------------------------------------------------------
Current         10.0     10     1062.1   IDR 13.2    M
Optimal         3.0      29     318.6    IDR 39.5    M
Balanced        7.0      13     743.5    IDR 18.1    M
Frequent        5.0      18     531.1    IDR 24.7    M
Cost_Efficient  12.0     7      1274.5   IDR 9.9     M

TOP 5 PASSENGER HOTSPOTS:
   1. PETUKANGAN JAKARTA: 201,978/month
   2. PURI BETA 2: 184,134/month
   3. CSW: 163,785/month
   4. TEGAL MAMPANG: 110,579/month
   5. VELBAK: 97,711/month

TOP 3 NEW STOP RECOMMENDATIONS:
   1. Between PETUKANGAN JAKARTA - PURI BETA 1
      Potential: 84,398 passengers/month
   2. Between PURI BETA 1 - PURI BETA 2
      Potential: 79,045 passengers/month
   3. Between PURI BETA 2